#### Business Analytics FHDW 2025
## Aufgabe

Betrachten Sie die Datei *groessen.csv*, die Stichproben der Körpergrößen von Sportlern verschiedener Disziplinen enthält. Führen Sie eine Varianzanalyse des Einflussfaktors *Sportart* inklusive einer ANOVA-Tabelle durch.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as sta

groessen_df = pd.read_csv('Daten/groessen.csv')
groessen_df

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))
ax.boxplot(groessen_df[[p for p in groessen_df.columns]])
plt.xticks(ticks=[1,2,3], labels=groessen_df.columns)
ax.set_ylabel('Größe')
plt.show()

In [ ]:
gesamtmittel = groessen_df.mean().sum()/len(groessen_df.mean())

mittelwerte_df = pd.DataFrame(columns=['Gruppenmittelwert','Wahrer Effekt'], 
                              index=['Turnen','Fussball','Volleyball'])
mittelwerte_df['Gruppenmittelwert'] = [groessen_df[p].mean() for p in groessen_df.columns]
mittelwerte_df['Wahrer Effekt'] = [gruppenmittel-gesamtmittel for gruppenmittel in mittelwerte_df.Gruppenmittelwert]
mittelwerte_df

In [ ]:
abweichungsquadrate_df = pd.DataFrame(columns=['SSt','SSb','SSw'],
                                      index=['Turnen','Fussball','Volleyball'])

abweichungsquadrate_df['SSt'] = [np.sum((groessen_df[kategorie]-gesamtmittel)**2) for kategorie in abweichungsquadrate_df.index]
abweichungsquadrate_df['SSb'] = [np.sum(len(groessen_df[kategorie])
                                        *(mittelwerte_df['Gruppenmittelwert'][kategorie]-gesamtmittel)**2) 
                                 for kategorie in abweichungsquadrate_df.index]
abweichungsquadrate_df['SSw'] = [np.sum((groessen_df[kategorie]-mittelwerte_df['Gruppenmittelwert'][kategorie])**2) 
                                 for kategorie in abweichungsquadrate_df.index]
SSt = np.sum(abweichungsquadrate_df['SSt'])
SSb = np.sum(abweichungsquadrate_df['SSb'])
SSw = np.sum(abweichungsquadrate_df['SSw'])

abweichungsquadrate_df.loc['Summe'] = [SSt, SSb, SSw]
abweichungsquadrate_df

In [ ]:
G = len(groessen_df.columns)
N = len(groessen_df.index)
df_ssb = G-1
df_ssw = G*(N-1)
MSb = SSb/df_ssb
MSw = SSw/df_ssw
F_emp = MSb/MSw

anova_df = pd.DataFrame({'SS':[SSb,SSw,SSt],
                         'df':[df_ssb,df_ssw,G*N-1],
                         'MS':[MSb,MSw,SSt/(G*N-1)]},
                        index=['Zwischen Faktorstufen','Innerhalb Faktorstufen','Gesamt'])
anova_df.index.name = 'Varianzquelle'
anova_df

In [ ]:
eta_quadrat = SSb/SSt
print("Eta-Quadrat = {:.2f}%, {:.2f}% bleiben unerklärt.".format(eta_quadrat*100,(1-eta_quadrat)*100))

In [ ]:
F_alpha = sta.f.ppf(0.95, dfn=df_ssb, dfd=df_ssw)
print('F_emp = {:.2f}\nF_alpha = {:.2f}'.format(F_emp, F_alpha))

In [ ]:
p_value = (1 - sta.f.cdf(F_emp, dfn=df_ssb, dfd=df_ssw))
p_value

Damit gilt $F_{emp} > F_\alpha$ und wir verwerfen hier die Nullhypothese: Wir können davon ausgehen, dass die Sportart einen signifikanten Einfluss auf die Körpergröße der Sportler hat.

Das wird ebenso bestätigt durch den Wert $p<\alpha$.